In [1]:
import numpy as np
import pandas as pd

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)
#testloader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False)


In [4]:
 class Module2(nn.Module):
  def __init__(self):
    super(Module2, self).__init__()
    
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    self.fc4 = nn.Linear(10,3)

  def forward(self,y):  #z batch of list of 9 images
    y1 = self.pool(F.relu(self.conv1(y)))
    y1 = self.pool(F.relu(self.conv2(y1)))
    y1 = y1.view(-1, 16 * 5 * 5)

    y1 = F.relu(self.fc1(y1))
    y1 = F.relu(self.fc2(y1))
    y1 = F.relu(self.fc3(y1))
    y1 = self.fc4(y1)
    return y1 


In [10]:
optimizer_what = optim.RMSprop(what_net.parameters(), lr=0.001)#,nesterov=True)
 

In [12]:
what_net = Module2().double()
#what_net = what_net.to("cuda")
checkpoint = torch.load("what_net_20.pt",map_location=torch.device('cpu'))
what_net.load_state_dict(checkpoint['model_state_dict'])
optimizer_what.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
training_loss = checkpoint['training_loss']
test_loss = checkpoint['test_loss']

what_net.eval()

Module2(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)

In [19]:
pred = []
lbls = []
with torch.no_grad():
  for i, data in enumerate(trainloader, 0):
    inputs, labels= data
    lbls.append(labels.numpy())
    inputs = inputs.double()
    outputs = what_net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    pred.append(predicted.cpu().numpy())


In [21]:
lbls  = np.concatenate(lbls,axis=0)
pred = np.concatenate(pred,axis=0)

In [22]:
lbls.shape,pred.shape

((50000,), (50000,))

In [24]:
idxs = np.logical_or(np.logical_or(lbls==0,lbls==1),lbls ==2)

In [30]:
print("Accuracy on CIFAR10 0,1,2 classes is ",(np.sum(lbls[idxs] == pred[idxs]) / (len(lbls[idxs])) )*100 )

Accuracy on CIFAR10 0,1,2 classes 57.31333333333334
